<center><img src="logo.png" alt="Header" style="width: 800px;"/></center>

In [31]:
# modified by Rod Soto
#  导入库  import library
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display
import time
import threading
import inspect
import ctypes
import numpy as np

In [32]:
# Dogzilla drive library
from DOGZILLALib import DOGZILLA
g_dog = DOGZILLA()

# 颜色识别动作组  Color recognition action group

In [33]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Close': ("Close", "关闭"),
    'Red': ("Red", "红色"),
    'Green': ("Green", "绿色"),
    'Blue': ("Blue", "蓝色"),
    'Yellow': ("Yellow", "黄色"),
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [34]:
# 图像数据转化  Image data transformation
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [35]:
# 关闭线程  stop thread
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [36]:
# 创建摄像头显示组件  Create the camera display component
image_widget = widgets.Image(format='jpeg', width=640, height=480)

# 动态检测可用的摄像头设备并打开  Dynamically detect and open the available camera device
def get_available_camera():
    for device_id in range(10):  # Try the first 10 devices (usually enough)
        cap = cv2.VideoCapture(device_id)
        if cap.isOpened():
            cap.release()
            return device_id
    return -1

# Get the available camera ID
camera_id = get_available_camera()
if camera_id == -1:
    print('No available camera found. Please check your camera connections.')
else:
    # Open the camera
    image = cv2.VideoCapture(camera_id)
    image.set(3, 640)  # Set width
    image.set(4, 480)  # Set height
    image.set(5, 30)   # Set frame rate
    image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
    print(f'Camera {camera_id} opened successfully.')

# Make sure to release the camera resource when done
def release_camera():
    if image.isOpened():
        image.release()
        print('Camera released.')

# Register release function to run when notebook is interrupted or finished
import atexit
atexit.register(release_camera)


True

In [37]:
# 关闭摄像头 Close_Camera
button_Close_Camera = widgets.Button(  
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )


# 按键按下事件处理   Key press event processing
def on_button_close_camera(b):
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        b.icon = 'uncheck'
        stop_thread(thread1)
        image.release()
    
# 关联按键事件回调 Button event callbacks
button_Close_Camera.on_click(on_button_close_camera)

In [38]:
# 创建相关控制变量  
g_mode = 0
g_action = 0

In [39]:
# 创建存储HSV色域颜色分类数据的数组  Creates an array that stores the HSV gamut color classification data
color_lower = np.array([156, 43, 46])
color_upper = np.array([180, 255, 255])

In [40]:
# 颜色选择按钮配置  Color selection button configuration
# 关闭 Close
Closebutton = widgets.Button(         
    description=Name_widgets['Close'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 红色 Red
Redbutton = widgets.Button(       
    description=Name_widgets['Red'][g_ENABLE_CHINESE],
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='uncheck' )

# 绿色 Green
Greenbutton = widgets.Button(     
    description=Name_widgets['Green'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 蓝色 Blue
Bluebutton = widgets.Button(        
    description=Name_widgets['Blue'][g_ENABLE_CHINESE],         
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 黄色 Yellow
Yellowbutton = widgets.Button(        
    description=Name_widgets['Yellow'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

In [41]:
# 清除按键显示状态  Clear button display status
def ALL_Uncheck():
    Redbutton.icon = 'uncheck'
    Greenbutton.icon = 'uncheck'
    Bluebutton.icon = 'uncheck'
    Yellowbutton.icon = 'uncheck'

# 按键按下事件处理   Key press event processing
def on_button_clicked(b):
    global color_lower, color_upper, g_mode
    global g_action
    ALL_Uncheck()
    b.icon = 'check'
    with output:
        print("Button clicked:", b.description)
    if b.description == Name_widgets['Close'][g_ENABLE_CHINESE]:
        g_dog.action(0xff)
        g_mode = 0
        g_action = 0
        b.icon = 'uncheck'
    elif b.description == Name_widgets['Red'][g_ENABLE_CHINESE]:
        color_lower = np.array([0, 43, 46])
        color_upper = np.array([10, 255, 255])
        g_dog.action(0xff)
        g_mode = 1
        g_action = 1
    elif b.description == Name_widgets['Green'][g_ENABLE_CHINESE]:
        color_lower = np.array([35, 43, 46])
        color_upper = np.array([77, 255, 255])
        g_dog.action(0xff)
        g_mode = 1
        g_action = 2
    elif b.description == Name_widgets['Blue'][g_ENABLE_CHINESE]:
        color_lower=np.array([100, 43, 46])
        color_upper = np.array([124, 255, 255])
        g_dog.action(0xff)
        g_mode = 1
        g_action = 3
    elif b.description == Name_widgets['Yellow'][g_ENABLE_CHINESE]:
        color_lower = np.array([26, 43, 46])
        color_upper = np.array([34, 255, 255])
        g_dog.action(0xff)
        g_mode = 1
        g_action = 4


In [42]:
# 关联按键事件回调 Button event callbacks
Redbutton.on_click(on_button_clicked)
Greenbutton.on_click(on_button_clicked)
Bluebutton.on_click(on_button_clicked)
Yellowbutton.on_click(on_button_clicked)
Closebutton.on_click(on_button_clicked)

In [43]:
# 控制动作 control action
def action_handle(color_x, color_y, color_r, action):
    if color_r > 60 and (220 <= color_x <= 420) and (140 <= color_y <= 340):
        if action == 1: 
            g_dog.action(14) # 伸懒腰 Stretch
            time.sleep(.1)
        elif action == 2:
            g_dog.action(13)  # 招手 Wave_Hand
            time.sleep(.1)
        elif action == 3:
            g_dog.action(16)  # 左右摇摆 Swing
            time.sleep(.1)
        elif action == 4:
            g_dog.action(18) # 找食物 Foraging
            time.sleep(.1)

# 颜色动作任务  Color Action task
def Color_Action_Task():
    global color_lower, color_upper, g_mode 
    global g_action
    t_start = time.time()
    fps = 0
    while True:
        ret, frame = image.read()               
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,color_lower,color_upper)  
        mask = cv2.erode(mask,None,iterations=2)
        mask = cv2.dilate(mask,None,iterations=2)
        mask = cv2.GaussianBlur(mask,(3,3),0)     
        cnts = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2] 
        if g_mode == 1: # 按钮切换开关  button switch
            if len(cnts) > 0:
                cnt = max (cnts, key = cv2.contourArea)
                (color_x,color_y),color_radius = cv2.minEnclosingCircle(cnt)
                if color_radius > 10:
                    # 将检测到的颜色标记出来  Mark the detected color
                    cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)
                    action_handle(color_x, color_y, color_radius, g_action)
        fps = fps + 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS " + str(int(mfps)), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 3)
        image_widget.value = bgr8_to_jpeg(frame)



In [44]:
# 机器狗恢复默认姿态  DOGZILLA reverts to default
g_dog.reset()

In [45]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=Color_Action_Task)
thread1.setDaemon(True)
thread1.start()

output = widgets.Output()
box_btn = widgets.VBox([Redbutton, Greenbutton, Bluebutton, Yellowbutton, Closebutton, button_Close_Camera])
box_display = widgets.HBox([image_widget, box_btn, output])
display(box_display)

/tmp/ipykernel_10922/4079389022.py:3: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  thread1.setDaemon(True)
